# Data fetching for HbA1c
 Imputing mean for Hba1c missing values with latest available values for other features.

In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta

# Step 1: Define number of patients to limit the processing
num_patients = 10000

# Step 2: Establish DB Connection
# IMPORTANT: Replace with your actual database host, user, password, and database name
try:
    conn = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='Omkar@123',
        database='ehr_raw_data6_7'
    )
    if conn.is_connected():
        print("Successfully connected to the database!")
except mysql.connector.Error as err:
    print(f"Error connecting to the database: {err}")
    # Exit or handle the error appropriately if connection fails
    exit() # Exiting for this example, consider a more robust error handling

# Step 3: Load relevant data from the database
# The query is updated to fetch required columns and filter by the new date range.
query = """
SELECT patient_id, visit_date, age, gender, cci_score, test_name, result_value,
        bmi, blood_pressure_systolic, blood_pressure_diastolic, blood_glucose_level
FROM ehr_feature
WHERE visit_date BETWEEN '2023-08-01' AND '2025-07-31'
"""
df = pd.read_sql(query, conn)

# Close the database connection as soon as data is fetched
if 'conn' in locals() and conn.is_connected():
    conn.close()
    print("Database connection closed.")

# Ensure correct data types after loading from SQL
df['visit_date'] = pd.to_datetime(df['visit_date'])
# Use 'Int64' for integer column that might contain NaNs (nulls from DB)
df['cci_score'] = pd.to_numeric(df['cci_score'], errors='coerce').astype('Int64')
# Ensure other numerical columns are also appropriate types if needed
df['result_value'] = pd.to_numeric(df['result_value'], errors='coerce')
df['bmi'] = pd.to_numeric(df['bmi'], errors='coerce')
df['blood_pressure_systolic'] = pd.to_numeric(df['blood_pressure_systolic'], errors='coerce')
df['blood_pressure_diastolic'] = pd.to_numeric(df['blood_pressure_diastolic'], errors='coerce')
df['blood_glucose_level'] = pd.to_numeric(df['blood_glucose_level'], errors='coerce')


# Step 4: Limit to the first `num_patients`
# This ensures processing is limited to a specific number of unique patients.
df = df.sort_values('patient_id')
unique_patients = df['patient_id'].unique()[:num_patients]
df = df[df['patient_id'].isin(unique_patients)].copy()


# Step 5: Create monthly HbA1c pivot for the new date range (July 2023 - June 2025)
# Filter the DataFrame to include only 'HbA1c' test results.
hba1c_df = df[df['test_name'] == 'HbA1c'].copy()

# Create a 'year_month' column for pivoting, in 'monthname_YYYY' format
hba1c_df['year_month'] = hba1c_df['visit_date'].dt.strftime('%B_%Y').str.lower()

# Sort data to identify the most recent 'result_value' for each patient within each year_month.
hba1c_df = hba1c_df.sort_values(by=['patient_id', 'visit_date'], ascending=[True, False]) # Sort by visit_date descending to get latest

# Remove duplicates within each patient-year_month group, keeping the most recent
latest_hba1c_df = hba1c_df.drop_duplicates(subset=['patient_id', 'year_month'], keep='first')

# Select only the required columns for pivoting
latest_hba1c_df = latest_hba1c_df[['patient_id', 'year_month', 'result_value']]

# Pivot the table to get 'year_month' as columns and HbA1c results as values.
hba1c_pivot = latest_hba1c_df.pivot(index='patient_id', columns='year_month', values='result_value')

# Generate a list of all expected 'monthname_YYYY' months from August 2023 to July 2025
start_date = datetime(2023, 8, 1)
end_date = datetime(2025, 7, 31)
current_date = start_date
expected_months = []
while current_date <= end_date:
    expected_months.append(current_date.strftime('%B_%Y').lower()) # Changed format to monthname_YYYY_lower
    # Move to the next month
    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1)

# Reindex the pivot table to ensure all expected months are present, filling missing with NaN.
hba1c_pivot = hba1c_pivot.reindex(columns=expected_months)

# Rename the month columns to the desired format (hba1c_monthname_YYYY).
hba1c_pivot.columns = [f"hba1c_{col}" for col in hba1c_pivot.columns]

# Step 6: Extract other features by prioritizing the latest available month for each feature
print("Extracting other features by prioritizing latest available month for each feature...")

other_feature_cols_to_extract = [
    'age', 'gender', 'bmi', 'blood_pressure_systolic', 'blood_pressure_diastolic',
    'blood_glucose_level', 'cci_score'
]

# Create an empty DataFrame to store the final other features
final_other_features = pd.DataFrame({'patient_id': unique_patients})

# Sort the main DataFrame by patient_id and visit_date (descending)
df_sorted_by_date = df.sort_values(by=['patient_id', 'visit_date'], ascending=[True, False])

# Group by patient_id and iterate to get the latest non-null value for each feature
grouped = df_sorted_by_date.groupby('patient_id')

for patient_id, group in grouped:
    patient_data = group.copy() # Make a copy to avoid SettingWithCopyWarning
    
    # Fill in each feature from the most recent date backwards
    for col in other_feature_cols_to_extract:
        # Get the first non-null value for the current column
        # This effectively checks rows from most recent to oldest
        value = patient_data[col].dropna().iloc[0] if not patient_data[col].dropna().empty else pd.NA
        final_other_features.loc[final_other_features['patient_id'] == patient_id, col] = value
        
print("Other feature extraction complete.")

# Step 7: Merge all processed dataframes
# Combine the monthly HbA1c data with the patient's other most recent features.
final_df = hba1c_pivot.merge(final_other_features, on='patient_id', how='left')

# Step 8: Impute missing values for HbA1c columns only
print("Imputing missing values for HbA1c columns...")

# Identify HbA1c columns
hba1c_cols = [col for col in final_df.columns if col.startswith('hba1c_')]

# Impute missing HbA1c values with the mean of available HbA1c values for that specific patient
for index, row in final_df.iterrows():
    patient_hba1c_values = row[hba1c_cols].dropna()
    if not patient_hba1c_values.empty:
        patient_hba1c_mean = round(patient_hba1c_values.mean(),2)
        for col in hba1c_cols:
            if pd.isna(row[col]):
                final_df.loc[index, col] = patient_hba1c_mean

print("Missing value imputation for HbA1c columns complete.")


# Step 9: Ensure the final output CSV has the specified column order
# Define the order of 'other features' columns explicitly
other_feature_cols = [
    'age', 'gender', 'bmi', 'blood_pressure_systolic', 'blood_pressure_diastolic',
    'blood_glucose_level', 'cci_score'
]

# Get the dynamically generated hba1c columns (re-get after potential imputation)
hba1c_cols = [col for col in final_df.columns if col.startswith('hba1c_')]

# Define the final desired column order
final_column_order = ['patient_id'] + other_feature_cols + hba1c_cols

# Reindex the DataFrame to enforce the column order
final_df = final_df[final_column_order]

# Step 10: Save the final processed data to a CSV file
final_df.to_csv("10k_Imputed_Transformed_hba1c_data6_7.csv", index=False)
print(f"Final data saved with shape: {final_df.shape}")

# Without Fetching Data


In [ ]:
import pandas as pd
from datetime import datetime

# Step 1: Define number of patients to limit the processing
num_patients = 10000

# Step 2: Load data from CSV file instead of SQL
try:
    df = pd.read_csv("10k_Imputed_Transformed_hba1c_data6_7.csv")
    print("Successfully loaded data from CSV file!")
except Exception as err:
    print(f"Error loading CSV file: {err}")
    exit()

# Ensure correct data types after loading from CSV
df['visit_date'] = pd.to_datetime(df['visit_date'])
# Use 'Int64' for integer column that might contain NaNs (nulls from DB)
df['cci_score'] = pd.to_numeric(df['cci_score'], errors='coerce').astype('Int64')
# Ensure other numerical columns are also appropriate types if needed
df['result_value'] = pd.to_numeric(df['result_value'], errors='coerce')
df['bmi'] = pd.to_numeric(df['bmi'], errors='coerce')
df['blood_pressure_systolic'] = pd.to_numeric(df['blood_pressure_systolic'], errors='coerce')
df['blood_pressure_diastolic'] = pd.to_numeric(df['blood_pressure_diastolic'], errors='coerce')
df['blood_glucose_level'] = pd.to_numeric(df['blood_glucose_level'], errors='coerce')

# Step 3: Limit to the first `num_patients`
# This ensures processing is limited to a specific number of unique patients.
df = df.sort_values('patient_id')
unique_patients = df['patient_id'].unique()[:num_patients]
df = df[df['patient_id'].isin(unique_patients)].copy()

# Step 4: Create monthly HbA1c pivot for the new date range (July 2023 - June 2025)
# Filter the DataFrame to include only 'HbA1c' test results.
hba1c_df = df[df['test_name'] == 'HbA1c'].copy()

# Create a 'year_month' column for pivoting, in 'monthname_YYYY' format
hba1c_df['year_month'] = hba1c_df['visit_date'].dt.strftime('%B_%Y').str.lower()

# Sort data to identify the most recent 'result_value' for each patient within each year_month.
hba1c_df = hba1c_df.sort_values(by=['patient_id', 'visit_date'], ascending=[True, False]) # Sort by visit_date descending to get latest

# Remove duplicates within each patient-year_month group, keeping the most recent
latest_hba1c_df = hba1c_df.drop_duplicates(subset=['patient_id', 'year_month'], keep='first')

# Select only the required columns for pivoting
latest_hba1c_df = latest_hba1c_df[['patient_id', 'year_month', 'result_value']]

# Pivot the table to get 'year_month' as columns and HbA1c results as values.
hba1c_pivot = latest_hba1c_df.pivot(index='patient_id', columns='year_month', values='result_value')

# Generate a list of all expected 'monthname_YYYY' months from August 2023 to July 2025
start_date = datetime(2023, 8, 1)
end_date = datetime(2025, 7, 31)
current_date = start_date
expected_months = []
while current_date <= end_date:
    expected_months.append(current_date.strftime('%B_%Y').lower()) # Changed format to monthname_YYYY_lower
    # Move to the next month
    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1)

# Reindex the pivot table to ensure all expected months are present, filling missing with NaN.
hba1c_pivot = hba1c_pivot.reindex(columns=expected_months)

# Rename the month columns to the desired format (hba1c_monthname_YYYY).
hba1c_pivot.columns = [f"hba1c_{col}" for col in hba1c_pivot.columns]

# Step 5: Extract other features by prioritizing the latest available month for each feature
print("Extracting other features by prioritizing latest available month for each feature...")

other_feature_cols_to_extract = [
    'age', 'gender', 'bmi', 'blood_pressure_systolic', 'blood_pressure_diastolic',
    'blood_glucose_level', 'cci_score'
]

# Create an empty DataFrame to store the final other features
final_other_features = pd.DataFrame({'patient_id': unique_patients})

# Sort the main DataFrame by patient_id and visit_date (descending)
df_sorted_by_date = df.sort_values(by=['patient_id', 'visit_date'], ascending=[True, False])

# Group by patient_id and iterate to get the latest non-null value for each feature
grouped = df_sorted_by_date.groupby('patient_id')

for patient_id, group in grouped:
    patient_data = group.copy() # Make a copy to avoid SettingWithCopyWarning
    
    # Fill in each feature from the most recent date backwards
    for col in other_feature_cols_to_extract:
        # Get the first non-null value for the current column
        # This effectively checks rows from most recent to oldest
        value = patient_data[col].dropna().iloc[0] if not patient_data[col].dropna().empty else pd.NA
        final_other_features.loc[final_other_features['patient_id'] == patient_id, col] = value
        
print("Other feature extraction complete.")

# Step 6: Merge all processed dataframes
# Combine the monthly HbA1c data with the patient's other most recent features.
final_df = hba1c_pivot.merge(final_other_features, on='patient_id', how='left')

# Step 7: Impute missing values for HbA1c columns only
print("Imputing missing values for HbA1c columns...")

# Identify HbA1c columns
hba1c_cols = [col for col in final_df.columns if col.startswith('hba1c_')]

# Impute missing HbA1c values with the mean of available HbA1c values for that specific patient
for index, row in final_df.iterrows():
    patient_hba1c_values = row[hba1c_cols].dropna()
    if not patient_hba1c_values.empty:
        patient_hba1c_mean = round(patient_hba1c_values.mean(),2)
        for col in hba1c_cols:
            if pd.isna(row[col]):
                final_df.loc[index, col] = patient_hba1c_mean

print("Missing value imputation for HbA1c columns complete.")

# Step 8: Ensure the final output CSV has the specified column order
# Define the order of 'other features' columns explicitly
other_feature_cols = [
    'age', 'gender', 'bmi', 'blood_pressure_systolic', 'blood_pressure_diastolic',
    'blood_glucose_level', 'cci_score'
]

# Get the dynamically generated hba1c columns (re-get after potential imputation)
hba1c_cols = [col for col in final_df.columns if col.startswith('hba1c_')]

# Define the final desired column order
final_column_order = ['patient_id'] + other_feature_cols + hba1c_cols

# Reindex the DataFrame to enforce the column order
final_df = final_df[final_column_order]

# Step 9: Return the final processed data
print(f"Final data processed with shape: {final_df.shape}")
final_df  # This will return the DataFrame

# Training and Testing/Validation for HbA1c
with quantile

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import pickle
import os
import calendar
from datetime import datetime, timedelta

# --- Configuration ---
DATA_FILE = "10k_Imputed_Transformed_hba1c_data6_7.csv"
MODELS_DIR = "HbA1c_Trained_Models_with_Quantiles_Data6_7" # Directory to save trained models
PREDICTIONS_ACTUAL_VS_PREDICTED_FILE = "HbA1c_2024-2025_Actual_vs_Predicted_Data6_7.csv"
TESTING_METRICS_FILE = "HbA1c_Testing_Metrics_Data6_7.csv"

# --- Define the base year for lag features and target horizons ---
# Adjust these years to easily change the timeframes
LAG_FEATURE_START_YEAR = 2023 # Lag features will start from August of this year
LAG_FEATURE_END_YEAR = 2024   # Lag features will end in July of this year

TARGET_HORIZON_START_YEAR = 2024 # Target horizons will start from August of this year
TARGET_HORIZON_END_YEAR = 2025   # Target horizons will end in July of this year


# Create directory for models if it doesn't exist
os.makedirs(MODELS_DIR, exist_ok=True)

# Load your dataset
df = pd.read_csv(DATA_FILE)

# --- NEW: Perform gender encoding ONCE on the full DataFrame before the loop ---
df['gender_encoded'] = df['gender'].astype('category').cat.codes

# Define static features for the model (now directly using 'gender_encoded')
static_features_for_model = [
    'age', 'gender_encoded', 'bmi', 'blood_pressure_systolic', 'blood_pressure_diastolic',
    'blood_glucose_level', 'cci_score'
]

# --- Dynamic generation of Lagged HbA1c features (August to July) ---
lag_features_for_training = []
# Loop for the starting year (August to December)
for month_num in range(8, 13):
    month_name = calendar.month_name[month_num].lower()
    lag_features_for_training.append(f"hba1c_{month_name}_{LAG_FEATURE_START_YEAR}")

# Loop for the ending year (January to July)
for month_num in range(1, 8):
    month_name = calendar.month_name[month_num].lower()
    lag_features_for_training.append(f"hba1c_{month_name}_{LAG_FEATURE_END_YEAR}")

# --- Dynamic generation of Target HbA1c columns (August to July) ---
target_cols = []
# Loop for the starting year (August to December)
for month_num in range(8, 13):
    month_name = calendar.month_name[month_num].lower()
    target_cols.append(f"hba1c_{month_name}_{TARGET_HORIZON_START_YEAR}")

# Loop for the ending year (January to July)
for month_num in range(1, 8):
    month_name = calendar.month_name[month_num].lower()
    target_cols.append(f"hba1c_{month_name}_{TARGET_HORIZON_END_YEAR}")


# Combine static and lag features to form the complete feature set for training
feature_cols_training = static_features_for_model + lag_features_for_training

# Handle missing values for features. Numeric NaNs are filled with 0.
df[feature_cols_training] = df[feature_cols_training].fillna(0)

# Initialize a DataFrame to store actual vs. predicted values for all patients
result_df_all_horizons = df[['patient_id']].copy()

# Lists to store training and testing metrics separately for console printing and saving
training_metrics_all_horizons = []
testing_metrics_all_horizons = []

# Function to calculate MAPE (Mean Absolute Percentage Error)
def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# --- Training and Saving Models for the defined horizons ---
print(f"--- Training Models and Saving to Disk ({TARGET_HORIZON_START_YEAR}-{TARGET_HORIZON_END_YEAR} Horizon) ---")
for h, target_col in enumerate(target_cols, start=1):
    # Skip if the target column is not present in the DataFrame
    if target_col not in df.columns:
        print(f"Warning: {target_col} missing in data, skipping this forecast horizon.")
        continue

    # Create a temporary DataFrame containing only non-null values for the current target.
    df_h = df.dropna(subset=[target_col]).copy()

    # Use the pre-defined feature_cols_training which already contains 'gender_encoded'.
    current_feature_cols = feature_cols_training

    # Define features (X) and target (y) for the current forecast horizon
    X = df_h[current_feature_cols]
    y = df_h[target_col]

    # Split the data into training and testing sets (80% training, 20% testing)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- Train and Save the main XGBoost Regressor model (your original logic) ---
    # This model uses 'reg:squarederror' and provides the point estimate.
    # We will continue to save it with its original filename.
    model_point = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model_point.fit(X_train, y_train)

    model_filename_point = os.path.join(MODELS_DIR, f"xgb_model_{target_col}.pkl")
    with open(model_filename_point, 'wb') as f:
        pickle.dump(model_point, f)
    print(f"Saved point prediction model for {target_col} to {model_filename_point}")


    # --- Train and Save the Lower Quantile XGBoost model ---
    # Using quantile_alpha=0.025 for the 2.5th percentile (for a 95% interval lower bound)
    model_lower_quantile = xgb.XGBRegressor(
        objective='reg:quantileerror',
        quantile_alpha=0.025,
        random_state=42,
        # Keep consistent hyperparameters with your main model
        n_estimators=model_point.n_estimators,
        learning_rate=model_point.learning_rate,
        max_depth=model_point.max_depth,
        # ... include any other hyperparameters from your model_point if set
    )
    model_lower_quantile.fit(X_train, y_train)

    model_filename_lower = os.path.join(MODELS_DIR, f"xgb_model_{target_col}_lower.pkl")
    with open(model_filename_lower, 'wb') as f:
        pickle.dump(model_lower_quantile, f)
    print(f"Saved lower quantile model for {target_col} to {model_filename_lower}")


    # --- Train and Save the Upper Quantile XGBoost model ---
    # Using quantile_alpha=0.975 for the 97.5th percentile (for a 95% interval upper bound)
    model_upper_quantile = xgb.XGBRegressor(
        objective='reg:quantileerror',
        quantile_alpha=0.975,
        random_state=42,
        # Keep consistent hyperparameters with your main model
        n_estimators=model_point.n_estimators,
        learning_rate=model_point.learning_rate,
        max_depth=model_point.max_depth,
        # ... include any other hyperparameters from your model_point if set
    )
    model_upper_quantile.fit(X_train, y_train)

    model_filename_upper = os.path.join(MODELS_DIR, f"xgb_model_{target_col}_upper.pkl")
    with open(model_filename_upper, 'wb') as f:
        pickle.dump(model_upper_quantile, f)
    print(f"Saved upper quantile model for {target_col} to {model_filename_upper}")


    # --- Evaluate Training Performance (using the point prediction model) ---
    y_train_pred = model_point.predict(X_train)
    y_train_pred = np.round(y_train_pred, 2)

    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mape_train = calculate_mape(y_train, y_train_pred)

    training_metrics_all_horizons.append({
        "Horizon_Month": h,
        "Target": target_col,
        "RMSE": rmse_train,
        "MAE": mae_train,
        "MAPE (%)": mape_train
    })

    # --- Evaluate Testing Performance (using the point prediction model) ---
    y_test_pred = model_point.predict(X_test)
    y_test_pred = np.round(y_test_pred, 2)

    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    mae_test = mean_absolute_error(y_test, y_test_pred)
    mape_test = calculate_mape(y_test, y_test_pred)

    testing_metrics_all_horizons.append({
        "Target": target_col,
        "RMSE": round(rmse_test,2),
        "MAE": round(mae_test,2),
        "MAPE (%)": round(mape_test,2)
    })

    testing_metrics_df = pd.DataFrame(testing_metrics_all_horizons)
    testing_metrics_df.to_csv(TESTING_METRICS_FILE, index=False)

    # --- Prepare data for actual vs. predicted CSV for current target (using the point prediction model) ---
    y_full_h_pred = model_point.predict(X)
    y_full_h_pred = np.round(y_full_h_pred, 2)

    result_df_all_horizons = result_df_all_horizons.merge(
        pd.DataFrame({
            'patient_id': df_h['patient_id'],
            f"actual_{target_col}": y,
            f"predicted_{target_col}": y_full_h_pred
        }),
        on='patient_id',
        how='left'
    )

print(f"\nTraining and Testing Performance ({TARGET_HORIZON_START_YEAR}-{TARGET_HORIZON_END_YEAR} Horizon) ---")
print("\nTraining Performance Across All Horizons ===")
for m in training_metrics_all_horizons:
    print(f"[{m['Target']}] RMSE: {m['RMSE']:.3f}, MAE: {m['MAE']:.3f}, MAPE: {m['MAPE (%)']:.2f}%")

print("\nTesting Performance Across All Horizons ===")
for m in testing_metrics_all_horizons:
    print(f"[{m['Target']}] RMSE: {m['RMSE']:.3f}, MAE: {m['MAE']:.3f}, MAPE: {m['MAPE (%)']:.2f}%")

# Save the actual vs. predicted results
result_df_all_horizons.to_csv(PREDICTIONS_ACTUAL_VS_PREDICTED_FILE, index=False)
print(f"\nActual vs. Predicted for {TARGET_HORIZON_START_YEAR}-{TARGET_HORIZON_END_YEAR} saved to: {PREDICTIONS_ACTUAL_VS_PREDICTED_FILE}")

print("\nTraining process complete: Point prediction models trained, and lower/upper quantile models trained and saved.")

--- Training Models and Saving to Disk (2024-2025 Horizon) ---
Saved point prediction model for hba1c_august_2024 to HbA1c_Trained_Models_with_Quantiles_Data6_7\xgb_model_hba1c_august_2024.pkl
Saved lower quantile model for hba1c_august_2024 to HbA1c_Trained_Models_with_Quantiles_Data6_7\xgb_model_hba1c_august_2024_lower.pkl
Saved upper quantile model for hba1c_august_2024 to HbA1c_Trained_Models_with_Quantiles_Data6_7\xgb_model_hba1c_august_2024_upper.pkl
Saved point prediction model for hba1c_september_2024 to HbA1c_Trained_Models_with_Quantiles_Data6_7\xgb_model_hba1c_september_2024.pkl
Saved lower quantile model for hba1c_september_2024 to HbA1c_Trained_Models_with_Quantiles_Data6_7\xgb_model_hba1c_september_2024_lower.pkl
Saved upper quantile model for hba1c_september_2024 to HbA1c_Trained_Models_with_Quantiles_Data6_7\xgb_model_hba1c_september_2024_upper.pkl
Saved point prediction model for hba1c_october_2024 to HbA1c_Trained_Models_with_Quantiles_Data6_7\xgb_model_hba1c_october_

# Prediction HbA1c 2025-2026

In [2]:
import pandas as pd
import numpy as np
import pickle
import os
import calendar
from datetime import datetime, timedelta

# --- Configuration for Prediction ---
DATA_FILE = "10k_Imputed_Transformed_hba1c_data6_7.csv"
MODELS_DIR = "HbA1c_Trained_Models_with_Quantiles_Data6_7" # Directory where trained models are saved
FINAL_OUTPUT_FILE = "HbA1c_2024-2026_Normalized_Data6_7.csv"

# --- Define the desired future prediction horizon ---
PREDICTION_FORECAST_START_YEAR = 2025
PREDICTION_FORECAST_END_YEAR = 2026

# --- Define the actuals historical period to include in output ---
ACTUALS_START_DATE = datetime(2024, 8, 1)
ACTUALS_END_DATE = datetime(2025, 7, 1) # Ends on the 1st of July 2025 for actuals for July 2025

# --- Month Names Helper ---
FULL_MONTH_NAMES = [calendar.month_name[i].lower() for i in range(1, 13)] # ['january', ..., 'december']

# --- Derived Years for Dynamic Column Generation ---
TRAINED_MODEL_TARGET_START_YEAR = PREDICTION_FORECAST_START_YEAR - 1
TRAINED_MODEL_TARGET_END_YEAR = PREDICTION_FORECAST_END_YEAR - 1

MODEL_TEMPLATE_LAG_START_YEAR = TRAINED_MODEL_TARGET_START_YEAR - 1
MODEL_TEMPLATE_LAG_END_YEAR = TRAINED_MODEL_TARGET_END_YEAR - 1

CURRENT_INPUT_LAG_START_YEAR = PREDICTION_FORECAST_START_YEAR - 1
CURRENT_INPUT_LAG_END_YEAR = PREDICTION_FORECAST_END_YEAR - 1


# Load your dataset
try:
    df = pd.read_csv(DATA_FILE)
except FileNotFoundError:
    print(f"Error: '{DATA_FILE}' not found. Please ensure the dataset file is in the specified path.")
    exit()


# Ensure gender is encoded consistently as it was during training
if 'gender' in df.columns and 'gender_encoded' not in df.columns:
    df['gender_encoded'] = df['gender'].astype('category').cat.codes
elif 'gender_encoded' not in df.columns:
    print("Warning: 'gender' column not found for encoding. Ensure 'gender_encoded' is present or handled.")


# Define static features (must match what models were trained on)
static_features_for_model = [
    'age', 'gender_encoded', 'bmi', 'blood_pressure_systolic', 'blood_pressure_diastolic',
    'blood_glucose_level', 'cci_score'
]

# --- Dynamic Generation of Lagged HbA1c features for the MODEL TEMPLATE ---
lag_features_model_template = (
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{MODEL_TEMPLATE_LAG_START_YEAR}" for month in range(8, 13)] + # Aug-Dec
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{MODEL_TEMPLATE_LAG_END_YEAR}" for month in range(1, 8)]      # Jan-Jul
)

# --- Dynamic Generation of Lagged HbA1c features for CURRENT PREDICTION INPUT ---
lag_features_current_input = (
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{CURRENT_INPUT_LAG_START_YEAR}" for month in range(8, 13)] + # Aug-Dec
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{CURRENT_INPUT_LAG_END_YEAR}" for month in range(1, 8)]      # Jan-Jul
)

# Combine static and lagged features to form the complete list of features for model input
feature_cols_for_model_input = static_features_for_model + lag_features_model_template


# Handle missing values for features used in prediction (fill with 0, consistent with training)
all_required_input_cols = static_features_for_model + lag_features_current_input
for col in all_required_input_cols:
    if col not in df.columns:
        print(f"Error: Required feature column '{col}' for prediction is missing in the input file '{DATA_FILE}'. Please ensure your data file is updated.")
        exit()

df[all_required_input_cols] = df[all_required_input_cols].fillna(0)


# --- Dynamic Generation of the target columns for the future prediction horizon ---
target_cols_future_prediction = (
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{PREDICTION_FORECAST_START_YEAR}" for month in range(8, 13)] + # Aug-Dec
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{PREDICTION_FORECAST_END_YEAR}" for month in range(1, 8)]      # Jan-Jul
)

# --- Dynamic Generation of the corresponding target columns that were used to train the models ---
trained_model_target_cols = (
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{TRAINED_MODEL_TARGET_START_YEAR}" for month in range(8, 13)] + # Aug-Dec
    [f"hba1c_{FULL_MONTH_NAMES[month-1]}_{TRAINED_MODEL_TARGET_END_YEAR}" for month in range(1, 8)]      # Jan-Jul
)


# Initialize a DataFrame to store the predictions temporarily in wide format
# This will be converted to long format later
temp_wide_predictions_df = df[['patient_id']].copy()

print(f"--- Generating Predictions for {PREDICTION_FORECAST_START_YEAR}-08 to {PREDICTION_FORECAST_END_YEAR}-07 Horizon ---")

# Iterate through the future target months and make predictions
for i, current_pred_target in enumerate(target_cols_future_prediction):
    model_to_load_name = trained_model_target_cols[i]
    
    # Define filenames for all three models (point, lower quantile, upper quantile)
    model_filename_point = os.path.join(MODELS_DIR, f"xgb_model_{model_to_load_name}.pkl")
    model_filename_lower = os.path.join(MODELS_DIR, f"xgb_model_{model_to_load_name}_lower.pkl")
    model_filename_upper = os.path.join(MODELS_DIR, f"xgb_model_{model_to_load_name}_upper.pkl")

    # Define output column names for point prediction and its bounds in the temp wide df
    output_col_name_point = f"predicted_{current_pred_target}"
    output_col_name_lower = f"predicted_{current_pred_target}_lower"
    output_col_name_upper = f"predicted_{current_pred_target}_upper"

    # Initialize columns to NaN in case any model or prediction fails for this month
    temp_wide_predictions_df[output_col_name_point] = np.nan
    temp_wide_predictions_df[output_col_name_lower] = np.nan
    temp_wide_predictions_df[output_col_name_upper] = np.nan

    # Initialize loaded models
    loaded_model_point = None
    loaded_model_lower = None
    loaded_model_upper = None

    try:
        # Load the point prediction model
        if os.path.exists(model_filename_point):
            with open(model_filename_point, 'rb') as f:
                loaded_model_point = pickle.load(f)
            print(f"Loaded point prediction model '{os.path.basename(model_filename_point)}'.")
        else:
            print(f"Error: Point prediction model file '{model_filename_point}' not found. Cannot predict '{current_pred_target}'. Skipping this month.")
            continue # Skip to the next target month if the primary model is missing

        # Load the lower quantile model
        if os.path.exists(model_filename_lower):
            with open(model_filename_lower, 'rb') as f:
                loaded_model_lower = pickle.load(f)
            print(f"Loaded lower quantile model '{os.path.basename(model_filename_lower)}'.")
        else:
            print(f"Warning: Lower quantile model file '{model_filename_lower}' not found. Lower bound for '{current_pred_target}' will be NaN.")

        # Load the upper quantile model
        if os.path.exists(model_filename_upper):
            with open(model_filename_upper, 'rb') as f:
                loaded_model_upper = pickle.load(f)
            print(f"Loaded upper quantile model '{os.path.basename(model_filename_upper)}'.")
        else:
            print(f"Warning: Upper quantile model file '{model_filename_upper}' not found. Upper bound for '{current_pred_target}' will be NaN.")

        # Prepare the input features for prediction.
        X_predict_data = df[static_features_for_model].copy()

        # Create a mapping for the lagged features from their current input names to their model template names
        lag_feature_mapping = {
            lag_current: lag_template
            for lag_current, lag_template in zip(lag_features_current_input, lag_features_model_template)
        }

        # Select the current input lagged features and rename them to match the model's expected input
        X_predict_lags = df[lag_features_current_input].rename(columns=lag_feature_mapping)

        # Combine static features and renamed lagged features
        X_predict = pd.concat([X_predict_data, X_predict_lags], axis=1)

        # Ensure the order of columns in X_predict matches the original training order
        X_predict = X_predict[feature_cols_for_model_input]

        # --- Make predictions using all three loaded models ---
        # Point prediction (values are rounded here to 2 decimal places)
        y_pred_point = loaded_model_point.predict(X_predict)
        y_pred_point = np.round(y_pred_point, 2)
        temp_wide_predictions_df[output_col_name_point] = y_pred_point

        # Lower bound prediction
        if loaded_model_lower:
            y_pred_lower = loaded_model_lower.predict(X_predict)
            y_pred_lower = np.round(y_pred_lower, 2) # Ensure rounding at this stage
            # Post-processing for lower bound:
            # 1. HbA1c cannot be negative.
            # 2. Lower bound should not cross above the point prediction.
            y_pred_lower = np.maximum(0, np.minimum(y_pred_lower, y_pred_point))
            temp_wide_predictions_df[output_col_name_lower] = y_pred_lower
        # Else: it's already NaN from initialization

        # Upper bound prediction
        if loaded_model_upper:
            y_pred_upper = loaded_model_upper.predict(X_predict)
            y_pred_upper = np.round(y_pred_upper, 2) # Ensure rounding at this stage
            # Post-processing for upper bound:
            # 1. Upper bound should not cross below the point prediction.
            y_pred_upper = np.maximum(y_pred_upper, y_pred_point)
            temp_wide_predictions_df[output_col_name_upper] = y_pred_upper
        # Else: it's already NaN from initialization

    except Exception as e:
        print(f"An unexpected error occurred while predicting '{current_pred_target}': {e}. All predictions for this month will be NaN.")
        continue # Skip to the next target month

print(f"\nCompleted generating wide-format predictions. Now transforming to long format...")

# --- Part 1: Extract Actuals in Long Format (2024-08-01 to 2025-07-31) ---
actual_cols_to_melt = []
current_date = ACTUALS_START_DATE
while current_date <= ACTUALS_END_DATE:
    month_name = FULL_MONTH_NAMES[current_date.month - 1]
    col_name = f"hba1c_{month_name}_{current_date.year}"
    if col_name in df.columns:
        actual_cols_to_melt.append(col_name)
    else:
        print(f"Warning: Actual column '{col_name}' missing from input data. It will not be included in the output actuals.")
    
    # Move to the next month
    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1)

actuals_df_long = pd.DataFrame()
if actual_cols_to_melt:
    # Melt the actual columns
    actuals_df_long = df[['patient_id'] + actual_cols_to_melt].melt(
        id_vars=['patient_id'],
        var_name='month_year_hba1c',
        value_name='value'
    )
    actuals_df_long['metric_type'] = 'actual'

    # Extract month and year from 'month_year_hba1c' to create 'Date' column in 'YYYY-MM-DD' format
    actuals_df_long['Date'] = actuals_df_long['month_year_hba1c'].apply(lambda x: datetime.strptime(x.split('_')[-2].capitalize() + x.split('_')[-1], '%B%Y').strftime('%Y-%m-%d'))
    actuals_df_long = actuals_df_long[['patient_id', 'Date', 'metric_type', 'value']] # Changed 'month' to 'Date' here
    # Round actual values to 2 decimal places as well for consistency in the final output
    actuals_df_long['value'] = actuals_df_long['value'].round(2)
    print(f"Extracted {len(actuals_df_long)} actuals records.")
else:
    print("No actuals columns found for the specified period. Actuals will not be included.")


# --- Part 2: Convert Predicted Data to Long Format ---
all_predicted_rows_list = []

# Loop through each predicted target month
for current_pred_target in target_cols_future_prediction:
    output_col_name_point = f"predicted_{current_pred_target}"
    output_col_name_lower = f"predicted_{current_pred_target}_lower"
    output_col_name_upper = f"predicted_{current_pred_target}_upper"

    # Extract month and year from the target column name
    parts = current_pred_target.split('_')
    month_name = parts[1]
    year = int(parts[2])
    month_num = FULL_MONTH_NAMES.index(month_name) + 1
    date_str = f"{year}-{month_num:02d}-01" # This will be the 'Date' value

    # Get the data for the current month across all patients
    temp_data = temp_wide_predictions_df[['patient_id', output_col_name_point, output_col_name_lower, output_col_name_upper]].copy()

    # Rename columns for melting
    temp_data.rename(columns={
        output_col_name_point: 'predicted',
        output_col_name_lower: 'lower_bound',
        output_col_name_upper: 'upper_bound'
    }, inplace=True)

    # Melt this month's data
    melted_month_data = temp_data.melt(
        id_vars=['patient_id'],
        var_name='metric_type',
        value_name='value'
    )
    melted_month_data['Date'] = date_str # Changed 'month' to 'Date' here
    
    # Append to the list
    all_predicted_rows_list.append(melted_month_data)

predictions_df_long = pd.DataFrame()
if all_predicted_rows_list:
    predictions_df_long = pd.concat(all_predicted_rows_list, ignore_index=True)
    print(f"Generated {len(predictions_df_long)} predicted records.")
else:
    print("No predictions were generated. Predicted data will not be included.")


# --- Part 3: Combine and Save the Final Output ---
final_output_df = pd.concat([actuals_df_long, predictions_df_long], ignore_index=True)

# Convert 'Date' column to datetime for proper chronological sorting
final_output_df['Date'] = pd.to_datetime(final_output_df['Date']) # Changed 'month' to 'Date' here

# Define the custom order for 'metric_type' as requested
metric_type_order = ['actual', 'predicted', 'lower_bound', 'upper_bound']
final_output_df['metric_type'] = pd.Categorical(final_output_df['metric_type'], categories=metric_type_order, ordered=True)

# Sort the DataFrame by patient_id, Date, and then the custom metric_type order
final_output_df.sort_values(by=['patient_id', 'Date', 'metric_type'], inplace=True) # Changed 'month' to 'Date' here

# Convert 'Date' column back to string format 'YYYY-MM-DD'
final_output_df['Date'] = final_output_df['Date'].dt.strftime('%Y-%m-%d') # Changed 'month' to 'Date' here

# Save the final DataFrame to CSV
final_output_df.to_csv(FINAL_OUTPUT_FILE, index=False, float_format="%.2f")
print(f"\nFinal combined actuals and predictions saved to: {FINAL_OUTPUT_FILE}")

print("\nOutput generation complete.")

--- Generating Predictions for 2025-08 to 2026-07 Horizon ---
Loaded point prediction model 'xgb_model_hba1c_august_2024.pkl'.
Loaded lower quantile model 'xgb_model_hba1c_august_2024_lower.pkl'.
Loaded upper quantile model 'xgb_model_hba1c_august_2024_upper.pkl'.
Loaded point prediction model 'xgb_model_hba1c_september_2024.pkl'.
Loaded lower quantile model 'xgb_model_hba1c_september_2024_lower.pkl'.
Loaded upper quantile model 'xgb_model_hba1c_september_2024_upper.pkl'.
Loaded point prediction model 'xgb_model_hba1c_october_2024.pkl'.
Loaded lower quantile model 'xgb_model_hba1c_october_2024_lower.pkl'.
Loaded upper quantile model 'xgb_model_hba1c_october_2024_upper.pkl'.
Loaded point prediction model 'xgb_model_hba1c_november_2024.pkl'.
Loaded lower quantile model 'xgb_model_hba1c_november_2024_lower.pkl'.
Loaded upper quantile model 'xgb_model_hba1c_november_2024_upper.pkl'.
Loaded point prediction model 'xgb_model_hba1c_december_2024.pkl'.
Loaded lower quantile model 'xgb_model_hb